In [4]:
import torch
import random
import os
import json
import numpy as np
import pandas as pd
from datetime import datetime
from argparse import ArgumentParser
from tensorboardX import SummaryWriter
import sys
from models.factorizer import setup_factorizer
from data_loader.data_loader import setup_generator
from utils.evaluate import evaluate_fm
import warnings


In [5]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.backends.cudnn.enabled = True


In [6]:

def setup_args(parser=None):
    """ Set up arguments for the Engine

    return:
        python dictionary
    """
    if parser is None:
        parser = ArgumentParser()
    data = parser.add_argument_group('Data')
    engine = parser.add_argument_group('Engine Arguments')
    factorize = parser.add_argument_group('Factorizer Arguments')
    matrix_factorize = parser.add_argument_group('MF Arguments')
    regularize = parser.add_argument_group('Regularizer Arguments')
    log = parser.add_argument_group('Tensorboard Arguments')

    engine.add_argument('--alias', default='experiment',
                        help='Name for the experiment')
    engine.add_argument('--seed', default='42')

    data.add_argument('--data-type', default='ml1m', help='type of the dataset')
    data.add_argument('--data-path', default='./data/{data_type}/')
    data.add_argument('--train_test-freq-bd', help='split the data freq-wise, bound of the user freq')
    data.add_argument('--train-valid-freq-bd', help='split the data freq-wise, bound of the user freq')
    data.add_argument('--batch-size-train', default=1)
    data.add_argument('--batch-size-valid', default=1)
    data.add_argument('--batch-size-test', default=1)
    data.add_argument('--device-ids-test', default=[0], help='devices used for multi-processing evaluate')

    regularize.add_argument('--max-steps', default=1e5)
    regularize.add_argument('--max-epochs', default=5)
    regularize.add_argument('--use-cuda', default=True)
    regularize.add_argument('--device-id', default=0, help='Training Devices')

    factorize.add_argument('--factorizer', default='fm', help='Type of the Factorization Model')
    factorize.add_argument('--latent-dim', default=8)

    type_opt = 'fm'
    matrix_factorize.add_argument('--{}-optimizer'.format(type_opt), default='sgd')
    matrix_factorize.add_argument('--{}-lr'.format(type_opt), default=1e-3)
    matrix_factorize.add_argument('--{}-grad-clip'.format(type_opt), default=1)

    log.add_argument('--log-interval', default=1)
    log.add_argument('--tensorboard', default='./tmp/runs')
    log.add_argument('--early_stop', default=None)
    log.add_argument('--display_interval', default=100)
    return parser





In [7]:
class Engine(object):
    """Engine wrapping the training & evaluation
       of adpative regularized maxtirx factorization
    """
    _global_writer = None  # Class variable for the global writer
    _param_step = 0  # Class variable to track parameter steps


    def __init__(self, opt):
        self._opt = opt
        self._opt['data_path'] = self._opt['data_path'].format(data_type=self._opt['data_type'])
        self._sampler = setup_generator(opt)

        self._opt['field_dims'] = self._sampler.field_dims

        if Engine._global_writer is None:
            Engine._global_writer = SummaryWriter(log_dir='{}/parameter_comparison'.format(self._opt['tensorboard']))
        

        self._opt['emb_save_path'] = self._opt['emb_save_path'].format(
            factorizer=self._opt['factorizer'],
            data_type=self._opt['data_type'],
            alias=self._opt['alias'],
            num_parameter='{num_parameter}'
        )
        if 'retrain_emb_param' in opt:
            self.retrain = True
            if opt['re_init']:
                self._opt['alias'] += '_reinitTrue'
            else:
                self._opt['alias'] += '_reinitFalse'
            self._opt['alias'] += '_retrain_emb_param{}'.format(opt['retrain_emb_param'])
        else:
            self.retrain = False
            self.candidate_p = self._opt.get('candidate_p')
        self._opt['eval_res_path'] = self._opt['eval_res_path'].format(
            factorizer=self._opt['factorizer'],
            data_type=self._opt['data_type'],
            alias=self._opt['alias'],
            epoch_idx='{epoch_idx}'
        )
        self._factorizer = setup_factorizer(opt)
        self._opt['tensorboard'] = self._opt['tensorboard'].format(
            factorizer=self._opt['factorizer'],
            data_type=self._opt['data_type'],
        )
        self._writer = SummaryWriter(log_dir='{}/{}'.format(self._opt['tensorboard'], opt['alias']))
        self._writer.add_text('option', str(opt), 0)
        self._mode = None
        self.early_stop = self._opt.get('early_stop')


    @property
    def mode(self):
        return self._mode

    @mode.setter
    def mode(self, new_mode):
        assert new_mode in ['complete', 'partial', None]  # training a complete trajectory or a partial trajctory
        self._mode = new_mode

    def save_pruned_embedding(self, param, step_idx):
        max_candidate_p = max(self.candidate_p)
        if max_candidate_p == 0:

            print("Minimal target parameters achieved, stop pruning.")
            return 0

        else:
            if param <= max_candidate_p:
                embedding = self._factorizer.model.get_embedding()
                emb_save_path = self._opt['emb_save_path'].format(num_parameter=param)
                emb_save_dir, _ = os.path.split(emb_save_path)
                if not os.path.exists(emb_save_dir):
                    os.makedirs(emb_save_dir)
                np.save(emb_save_path, embedding)
                max_idx = self.candidate_p.index(max(self.candidate_p))
                self.candidate_p[max_idx] = 0
                print("*" * 80)
                print("Reach the target parameter: {}, save embedding with size: {}".format(max_candidate_p, param))
                print("*" * 80)
            elif step_idx == 0:
                embedding = self._factorizer.model.get_embedding()
                emb_save_path = self._opt['emb_save_path'].format(num_parameter='initial_embedding')
                emb_save_dir, _ = os.path.split(emb_save_path)
                if not os.path.exists(emb_save_dir):
                    os.makedirs(emb_save_dir)
                np.save(emb_save_path, embedding)
                print("*" * 80)
                print("Save the initial embedding table")
                print("*" * 80)
        return 1
    
    def train_an_episode(self, max_steps, max_epochs, episode_idx=''):
        """Train a feature_based recommendation model"""
        assert self.mode in ['partial', 'complete']

        print('-' * 80)
        print('[{} episode {} starts!]'.format(self.mode, episode_idx))
        print('Initializing ...')
        self._factorizer.init_episode()

        log_interval = self._opt.get('log_interval')
        eval_interval = self._opt.get('eval_interval')
        display_interval = self._opt.get('display_interval')

        status = dict()
        flag, test_flag, valid_flag = 0, 0, 0
        valid_mf_loss, train_mf_loss = np.inf, np.inf
        best_valid_result = {"AUC": [0, 0], "LogLoss": [np.inf, 0]}
        best_test_result = {"AUC": [0, 0], "LogLoss": [np.inf, 0]}
        epoch_start = datetime.now()
        for step_idx in range(int(max_steps)):
            # Prepare status for current step
            status['done'] = False
            status['sampler'] = self._sampler
            train_mf_loss = self._factorizer.update(self._sampler)
            status['train_mf_loss'] = train_mf_loss

            # Logging & Evaluate on the Evaluate Set
            if self.mode == 'complete' and step_idx % log_interval == 0:
                epoch_idx = int(step_idx / self._sampler.num_batches_train)
                if epoch_idx > max_epochs and self.retrain:
                    return best_test_result

                sparsity, params = self._factorizer.model.calc_sparsity()
                if not self.retrain:
                    returnflag = self.save_pruned_embedding(params, step_idx)
                    if not returnflag:
                        return best_test_result
                    
                self._writer.add_scalar('train/step_wise/mf_loss', train_mf_loss, step_idx)
                self._writer.add_scalar('train/step_wise/sparsity', sparsity, step_idx)

                if step_idx % display_interval == 0:
                    print('[Epoch {}|Step {}|Flag {}|Sparsity {:.4f}|Params {}| Best AUC {}]'.format(epoch_idx,
                                                                                        step_idx % self._sampler.num_batches_train,
                                                                                        flag, sparsity, params,
                                                                                        round(best_test_result['AUC'][0],4)))

                if step_idx % self._sampler.num_batches_train == 0:
                    threshold = self._factorizer.model.get_threshold()

                    self._writer.add_histogram('threshold/epoch_wise/threshold', threshold, epoch_idx)
                    self._writer.add_scalar('train/epoch_wise/sparsity', sparsity, epoch_idx)
                    self._writer.add_scalar('train/epoch_wise/params', params, epoch_idx)

                if (step_idx % self._sampler.num_batches_train == 0) and (epoch_idx % eval_interval == 0) and self.retrain:
                    print('Evaluate on test ...')
                    start = datetime.now()
                    eval_res_path = self._opt['eval_res_path'].format(epoch_idx=epoch_idx)
                    eval_res_dir, _ = os.path.split(eval_res_path)
                    if not os.path.exists(eval_res_dir):
                        os.makedirs(eval_res_dir)

                    use_cuda = self._opt['use_cuda']
                    logloss, auc = evaluate_fm(self._factorizer, self._sampler, use_cuda)
                    self._writer.add_scalar('test/epoch_wise/metron_auc', auc, epoch_idx)
                    self._writer.add_scalar('test/epoch_wise/metron_logloss', logloss, epoch_idx)
                    if logloss < best_test_result['LogLoss'][0]:
                        best_test_result['LogLoss'][0] = logloss
                        best_test_result['LogLoss'][1] = epoch_idx
                    if auc > best_test_result['AUC'][0]:
                        best_test_result['AUC'][0] = auc
                        best_test_result['AUC'][1] = epoch_idx
                        test_flag = 0
                    else:
                        test_flag += 1
                    pd.Series(best_test_result).to_csv(eval_res_path)
                    print("*" * 80)
                    print("Test AUC: {:4f} | Logloss: {:4f}".format(auc, logloss))
                    end = datetime.now()
                    print('Evaluate Time {} minutes'.format((end - start).total_seconds() / 60))
                    epoch_end = datetime.now()
                    dur = (epoch_end - epoch_start).total_seconds() / 60
                    epoch_start = datetime.now()
                    print('[Epoch {:4d}] train MF loss: {:04.8f}, '
                          'valid loss: {:04.8f}, time {:04.8f} minutes'.format(epoch_idx,
                                                                               train_mf_loss,
                                                                               valid_mf_loss,
                                                                               dur))
                    print("*"*80)
                    
            flag = test_flag
            if self.early_stop is not None and flag >= self.early_stop :
                print("Early stop training process")
                print("Best performance on test data: ", best_test_result)
                print("Best performance on valid data: ", best_valid_result)
                self._writer.add_text('best_valid_result', str(best_valid_result), 0)
                self._writer.add_text('best_test_result', str(best_test_result), 0)
                return best_test_result
        return best_test_result
                
            
            
    def train_finish(self, best_test_result):
        Engine._global_writer.add_scalar(
            'parameter_comparison/best_auc_vs_params', 
            best_test_result['AUC'][0],
            Engine._param_step
        )
        Engine._param_step += 1  # Increment the step counter
        Engine._global_writer.flush()

    def __del__(self):
        if hasattr(self, '_writer'):
            self._writer.close()
        if Engine._global_writer is not None:  # Close global writer if exists
            Engine._global_writer.close()





    def train(self):
        self.mode = 'complete'
        best_test_result = self.train_an_episode(self._opt['max_steps'],self._opt['max_epochs'])
        return best_test_result




In [8]:
torch.cuda.is_available()

True

# Train FM, with avazu Dataset

In [9]:

torch.cuda.empty_cache()


parser = setup_args()
parser.set_defaults(
    alias='train',
    tensorboard='./tmp/runs/{factorizer}/{data_type}',
    ##########
    ## data ##
    ##########
    data_type='avazu',
    data_path='./data/{data_type}/',
    load_in_queue=False,
    category_only=False,
    rebuild_cache=False,
    eval_res_path='./tmp/res/{factorizer}/{data_type}/{alias}/{epoch_idx}.csv',
    emb_save_path='./tmp/embedding/{factorizer}/{data_type}/{alias}/{num_parameter}',
    ######################
    ## train/test split ##
    ######################
    test_ratio=0.1,
    valid_ratio=1/9,
    ##########################
    ## Devices & Efficiency ##
    ##########################
    use_cuda=True,
    early_stop=10,
    log_interval=1,
    display_interval=1000,
    eval_interval=5,  # 10 epochs between 2 evaluations
    device_ids_test=[0],
    device_id=0,
    batch_size_train=1024,
    batch_size_valid=1024,
    batch_size_test=1024,
    ###########
    ## Model ##
    ###########
    factorizer='fm',
    model='fm',
    fm_lr=1e-3,
    # Deep
    mlp_dims=[100, 100],
    # AutoInt
    has_residual=True,
    full_part=True,
    num_heads=2,
    num_layers=3,
    att_dropout=0.4,
    atten_embed_dim=64,
    # optimizer setting
    fm_optimizer='adam',
    fm_amsgrad=False,
    fm_eps=1e-8,
    fm_l2_regularization=1e-5,
    fm_betas=(0.9, 0.999),
    fm_grad_clip=100,  # 0.1
    fm_lr_exp_decay=1,
    l2_penalty=0,
    #########
    ## Embeddings//PEP ##
    #########
    latent_dim=32,
    threshold_type='feature_dim',
    g_type='sigmoid',
    gk=1,
    threshold_init=-15,
    candidate_p=[50000, 30000],
)

opt = parser.parse_args(args=[])
opt = vars(opt)

# rename alias

opt['alias'] = '{}_{}_BaseDim{}_bsz{}_lr_{}_optim_{}_thresholdType{}_thres_init{}_{}-{}_l2_penalty{}'.format(
    opt['model'].upper(),
    opt['alias'],
    opt['latent_dim'],
    opt['batch_size_train'],
    opt['fm_lr'],
    opt['fm_optimizer'],
    opt['threshold_type'].upper(),
    opt['threshold_init'],
    opt['g_type'],
    opt['gk'],
    opt['l2_penalty']
)
print(opt['alias'])
random.seed(opt['seed'])
# np.random.seed(opt['seed'])
torch.manual_seed(opt['seed'])
torch.cuda.manual_seed_all(opt['seed'])
engine = Engine(opt)
best_result = engine.train()
print("*"*10 , best_result, "*"*10)



FM_train_BaseDim32_bsz1024_lr_0.001_optim_adam_thresholdTypeFEATURE_DIM_thres_init-15_sigmoid-1_l2_penalty0
	Num of train records: 32343173
	Num of valid records: 4042896
	Num of test records: 4042898
	Num of fields: 22
	Num of features: 645394


d:\pytorchEnv\env\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


BackBone Embedding Parameters:  20652608


d:\pytorchEnv\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--------------------------------------------------------------------------------
[complete episode  starts!]
Initializing ...
BackBone Embedding Parameters:  20652608
********************************************************************************
Save the initial embedding table
********************************************************************************
[Epoch 0|Step 0|Flag 0|Sparsity 0.0001|Params 20650466| Best AUC 0]
[Epoch 0|Step 1000|Flag 0|Sparsity 0.9389|Params 1262518| Best AUC 0]
[Epoch 0|Step 2000|Flag 0|Sparsity 0.9472|Params 1090701| Best AUC 0]
[Epoch 0|Step 3000|Flag 0|Sparsity 0.9508|Params 1016313| Best AUC 0]
[Epoch 0|Step 4000|Flag 0|Sparsity 0.9532|Params 966268| Best AUC 0]
[Epoch 0|Step 5000|Flag 0|Sparsity 0.9553|Params 922795| Best AUC 0]
[Epoch 0|Step 6000|Flag 0|Sparsity 0.9575|Params 876892| Best AUC 0]
[Epoch 0|Step 7000|Flag 0|Sparsity 0.9606|Params 813026| Best AUC 0]
[Epoch 0|Step 8000|Flag 0|Sparsity 0.9658|Params 706435| Best AUC 0]
[Epoch 0|Step 9

In [13]:
target_parameters = [ 29985, 50000]
best_test_results_list = []
for target_param in target_parameters:

    print(f"retrain for target parameters {target_param}")
    parser = setup_args()
    parser.set_defaults(
        alias='train',
        tensorboard='./tmp/runs/{factorizer}/{data_type}',
        ##########
        ## data ##
        ##########
        data_type='avazu',
        data_path='./data/{data_type}/',
        load_in_queue=False,
        category_only=False,
        rebuild_cache=False,
        eval_res_path='./tmp/res/{factorizer}/{data_type}/{alias}/{epoch_idx}.csv',
        emb_save_path='./tmp/embedding/{factorizer}/{data_type}/{alias}/{num_parameter}',
        ######################
        ## train/test split ##
        ######################
        test_ratio=0.1,
        valid_ratio=1/9,
        ##########################
        ## Devices & Efficiency ##
        ##########################
        use_cuda=True,
        early_stop=40,
        log_interval=1,
        display_interval=1000,
        eval_interval=5,  # 10 epochs between 2 evaluations
        device_ids_test=[0],
        device_id=0,
        batch_size_train=1024,
        batch_size_valid=1024,
        batch_size_test=1024,
        ###########
        ## Model ##
        ###########
        factorizer='fm',
        model='fm',
        fm_lr=1e-3,
        # Deep
        mlp_dims=[100, 100],
        # AutoInt
        has_residual=True,
        full_part=True,
        num_heads=2,
        num_layers=3,
        att_dropout=0.4,
        atten_embed_dim=64,
        # optimizer setting
        fm_optimizer='adam',
        fm_amsgrad=False,
        fm_eps=1e-8,
        fm_l2_regularization=1e-5,
        fm_betas=(0.9, 0.999),
        fm_grad_clip=100,  # 0.1
        fm_lr_exp_decay=1,
        l2_penalty=0,
        #########
        ## PEP ##
        #########
        latent_dim=32,
        threshold_type='feature_dim',
        g_type='sigmoid',
        gk=1,
        threshold_init=-15,
        retrain_emb_param=target_param,
        re_init=False,
    )

    opt = parser.parse_args(args=[])
    opt = vars(opt)
    opt['alias'] = '{}_{}_BaseDim{}_bsz{}_lr_{}_optim_{}_thresholdType{}_thres_init{}_{}-{}_l2_penalty{}'.format(
        opt['model'].upper(),
        opt['alias'],
        opt['latent_dim'],
        opt['batch_size_train'],
        opt['fm_lr'],
        opt['fm_optimizer'],
        opt['threshold_type'].upper(),
        opt['threshold_init'],
        opt['g_type'],
        opt['gk'],
        opt['l2_penalty']
    )
    print(opt['alias'])
    random.seed(opt['seed'])
    # np.random.seed(opt['seed'])
    torch.manual_seed(opt['seed'])
    torch.cuda.manual_seed_all(opt['seed'])
    engine = Engine(opt)
    best_result = engine.train()
    print("bbbbbbbbbbbbb",best_result)
    best_test_results_list.append(best_result)
    


retrain for target parameters 29985
FM_train_BaseDim32_bsz1024_lr_0.001_optim_adam_thresholdTypeFEATURE_DIM_thres_init-15_sigmoid-1_l2_penalty0
	Num of train records: 32343173
	Num of valid records: 4042896
	Num of test records: 4042898
	Num of fields: 22
	Num of features: 645394
Retrain epoch 29985
BackBone Embedding Parameters:  20652608
--------------------------------------------------------------------------------
[complete episode  starts!]
Initializing ...
Retrain epoch 29985
BackBone Embedding Parameters:  20652608
[Epoch 0|Step 0|Flag 0|Sparsity 0.9985|Params 29985| Best AUC 0]
Evaluate on test ...
********************************************************************************
Test AUC: 0.501445 | Logloss: 2.388995
Evaluate Time 10.274916916666665 minutes
[Epoch    0] train MF loss: 2.33896732, valid loss: 0inf, time 10.37719768 minutes
********************************************************************************
[Epoch 0|Step 1000|Flag 0|Sparsity 0.9985|Params 29985| Best

KeyboardInterrupt: 

In [ ]:
print(best_test_results_list)
avazu_FM_PEP = {}
for i, param in enumerate([29985, 50000]):
    avazu_FM_PEP[param] =  best_test_results_list[i]

pd.DataFrame(avazu_FM_PEP).to_csv("avazu_FM_PEP.csv")

[{'AUC': [0.7926537810981514, 15], 'LogLoss': [0.45323467, 15]}, {'AUC': [0.7919997853039459, 15], 'LogLoss': [0.4540726, 15]}, {'AUC': [0.7907308301215545, 15], 'LogLoss': [0.4555841, 15]}]


# Train DEEPFM, with avazu Dataset

In [10]:

torch.cuda.empty_cache()


parser = setup_args()
parser.set_defaults(
    alias='train',
    tensorboard='./tmp/runs/{factorizer}/{data_type}',
    ##########
    ## data ##
    ##########
    data_type='avazu',
    data_path='./data/{data_type}/',
    load_in_queue=False,
    category_only=False,
    rebuild_cache=False,
    eval_res_path='./tmp/res/{factorizer}/{data_type}/{alias}/{epoch_idx}.csv',
    emb_save_path='./tmp/embedding/{factorizer}/{data_type}/{alias}/{num_parameter}',
    ######################
    ## train/test split ##
    ######################
    test_ratio=0.1,
    valid_ratio=1/9,
    ##########################
    ## Devices & Efficiency ##
    ##########################
    use_cuda=True,
    early_stop=10,
    log_interval=1,
    display_interval=1000,
    eval_interval=5,  # 10 epochs between 2 evaluations
    device_ids_test=[0],
    device_id=0,
    batch_size_train=1024,
    batch_size_valid=1024,
    batch_size_test=1024,
    ###########
    ## Model ##
    ###########
    factorizer='fm',
    model='deepfm',
    fm_lr=1e-3,
    # Deep
    mlp_dims=[100, 100],
    # AutoInt
    has_residual=True,
    full_part=True,
    num_heads=2,
    num_layers=3,
    att_dropout=0.4,
    atten_embed_dim=64,
    # optimizer setting
    fm_optimizer='adam',
    fm_amsgrad=False,
    fm_eps=1e-8,
    fm_l2_regularization=1e-5,
    fm_betas=(0.9, 0.999),
    fm_grad_clip=100,  # 0.1
    fm_lr_exp_decay=1,
    l2_penalty=0,
    #########
    ## Embeddings//PEP ##
    #########
    latent_dim=32,
    threshold_type='feature_dim',
    g_type='sigmoid',
    gk=1,
    threshold_init=-15,
    candidate_p=[50000, 30000],
)

opt = parser.parse_args(args=[])
opt = vars(opt)

# rename alias

opt['alias'] = '{}_{}_BaseDim{}_bsz{}_lr_{}_optim_{}_thresholdType{}_thres_init{}_{}-{}_l2_penalty{}'.format(
    opt['model'].upper(),
    opt['alias'],
    opt['latent_dim'],
    opt['batch_size_train'],
    opt['fm_lr'],
    opt['fm_optimizer'],
    opt['threshold_type'].upper(),
    opt['threshold_init'],
    opt['g_type'],
    opt['gk'],
    opt['l2_penalty']
)
print(opt['alias'])
random.seed(opt['seed'])
# np.random.seed(opt['seed'])
torch.manual_seed(opt['seed'])
torch.cuda.manual_seed_all(opt['seed'])
engine = Engine(opt)
best_result = engine.train()
print("*"*10 , best_result, "*"*10)



DEEPFM_train_BaseDim32_bsz1024_lr_0.001_optim_adam_thresholdTypeFEATURE_DIM_thres_init-15_sigmoid-1_l2_penalty0
	Num of train records: 32343173
	Num of valid records: 4042896
	Num of test records: 4042898
	Num of fields: 22
	Num of features: 645394


d:\pytorchEnv\env\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


BackBone Embedding Parameters:  20652608
--------------------------------------------------------------------------------
[complete episode  starts!]
Initializing ...
BackBone Embedding Parameters:  20652608
********************************************************************************
Save the initial embedding table
********************************************************************************
[Epoch 0|Step 0|Flag 0|Sparsity 0.0001|Params 20650473| Best AUC 0]
[Epoch 0|Step 1000|Flag 0|Sparsity 0.8904|Params 2263960| Best AUC 0]
[Epoch 0|Step 2000|Flag 0|Sparsity 0.8969|Params 2129418| Best AUC 0]
[Epoch 0|Step 3000|Flag 0|Sparsity 0.9037|Params 1989730| Best AUC 0]
[Epoch 0|Step 4000|Flag 0|Sparsity 0.9112|Params 1833394| Best AUC 0]
[Epoch 0|Step 5000|Flag 0|Sparsity 0.9201|Params 1649551| Best AUC 0]
[Epoch 0|Step 6000|Flag 0|Sparsity 0.9305|Params 1434510| Best AUC 0]
[Epoch 0|Step 7000|Flag 0|Sparsity 0.9422|Params 1193746| Best AUC 0]
[Epoch 0|Step 8000|Flag 0|Sparsity 0.95

# Retrain for DeepFM

In [ ]:
target_parameters = [29989,49968]
best_test_results_list = []
for target_param in target_parameters:

    print(f"retrain for target parameters {target_param}")
    parser = setup_args()
    parser.set_defaults(
        alias='train',
        tensorboard='./tmp/runs/{factorizer}/{data_type}',
        ##########
        ## data ##
        ##########
        data_type='avazu',
        data_path='./data/{data_type}/',
        load_in_queue=False,
        category_only=False,
        rebuild_cache=False,
        eval_res_path='./tmp/res/{factorizer}/{data_type}/{alias}/{epoch_idx}.csv',
        emb_save_path='./tmp/embedding/{factorizer}/{data_type}/{alias}/{num_parameter}',
        ######################
        ## train/test split ##
        ######################
        test_ratio=0.1,
        valid_ratio=1/9,
        ##########################
        ## Devices & Efficiency ##
        ##########################
        use_cuda=True,
        early_stop=40,
        log_interval=1,
        display_interval=1000,
        eval_interval=5,  # 10 epochs between 2 evaluations
        device_ids_test=[0],
        device_id=0,
        batch_size_train=1024,
        batch_size_valid=1024,
        batch_size_test=1024,
        ###########
        ## Model ##
        ###########
        factorizer='fm',
        model='deepfm',
        fm_lr=1e-3,
        # Deep
        mlp_dims=[100, 100],
        # AutoInt
        has_residual=True,
        full_part=True,
        num_heads=2,
        num_layers=3,
        att_dropout=0.4,
        atten_embed_dim=64,
        # optimizer setting
        fm_optimizer='adam',
        fm_amsgrad=False,
        fm_eps=1e-8,
        fm_l2_regularization=1e-5,
        fm_betas=(0.9, 0.999),
        fm_grad_clip=100,  # 0.1
        fm_lr_exp_decay=1,
        l2_penalty=0,
        #########
        ## PEP ##
        #########
        latent_dim=32,
        threshold_type='feature_dim',
        g_type='sigmoid',
        gk=1,
        threshold_init=-15,
        retrain_emb_param=target_param,
        re_init=False,
    )

    opt = parser.parse_args(args=[])
    opt = vars(opt)
    opt['alias'] = '{}_{}_BaseDim{}_bsz{}_lr_{}_optim_{}_thresholdType{}_thres_init{}_{}-{}_l2_penalty{}'.format(
        opt['model'].upper(),
        opt['alias'],
        opt['latent_dim'],
        opt['batch_size_train'],
        opt['fm_lr'],
        opt['fm_optimizer'],
        opt['threshold_type'].upper(),
        opt['threshold_init'],
        opt['g_type'],
        opt['gk'],
        opt['l2_penalty']
    )
    print(opt['alias'])
    random.seed(opt['seed'])
    # np.random.seed(opt['seed'])
    torch.manual_seed(opt['seed'])
    torch.cuda.manual_seed_all(opt['seed'])
    engine = Engine(opt)
    best_result = engine.train()
    best_test_results_list.append(best_result)
    


retrain for target parameters 19998
DEEPFM_train_BaseDim32_bsz1024_lr_0.001_optim_adam_thresholdTypeFEATURE_DIM_thres_init-15_sigmoid-1_l2_penalty0
0
	Num of train records: 2400000
	Num of valid records: 300000
	Num of test records: 300000
	Num of fields: 39
	Num of features: 150032
Retrain epoch 19998
BackBone Embedding Parameters:  4801024
--------------------------------------------------------------------------------
[complete episode  starts!]
Initializing ...
Retrain epoch 19998
BackBone Embedding Parameters:  4801024
[Epoch 0|Step 0|Flag 0|Sparsity 0.9958|Params 19998| AUC [0, 0]]
Evaluate on test ...
********************************************************************************
Test AUC: 0.520811 | Logloss: 2.337113
Evaluate Time 0.08480715 minutes
[Epoch    0] train MF loss: 2.24896526, valid loss: 0inf, time 0.09538002 minutes
********************************************************************************
[Epoch 0|Step 1000|Flag 0|Sparsity 0.9958|Params 19998| AUC [0.52081

d:\pytorchEnv\env\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Retrain epoch 29984
BackBone Embedding Parameters:  4801024
--------------------------------------------------------------------------------
[complete episode  starts!]
Initializing ...
Retrain epoch 29984
BackBone Embedding Parameters:  4801024
[Epoch 0|Step 0|Flag 0|Sparsity 0.9938|Params 29984| AUC [0, 0]]
Evaluate on test ...
********************************************************************************
Test AUC: 0.520813 | Logloss: 2.337095
Evaluate Time 0.14400300000000002 minutes
[Epoch    0] train MF loss: 2.25208998, valid loss: 0inf, time 0.15328797 minutes
********************************************************************************
[Epoch 0|Step 1000|Flag 0|Sparsity 0.9938|Params 29984| AUC [0.520813484224496, 0]]
[Epoch 0|Step 2000|Flag 0|Sparsity 0.9938|Params 29984| AUC [0.520813484224496, 0]]
[Epoch 1|Step 656|Flag 0|Sparsity 0.9938|Params 29984| AUC [0.520813484224496, 0]]
[Epoch 1|Step 1656|Flag 0|Sparsity 0.9938|Params 29984| AUC [0.520813484224496, 0]]
[Epoch 2

d:\pytorchEnv\env\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


--------------------------------------------------------------------------------
[complete episode  starts!]
Initializing ...
Retrain epoch 49971
BackBone Embedding Parameters:  4801024
[Epoch 0|Step 0|Flag 0|Sparsity 0.9896|Params 49971| AUC [0, 0]]
Evaluate on test ...
********************************************************************************
Test AUC: 0.520813 | Logloss: 2.337081
Evaluate Time 0.09948191666666667 minutes
[Epoch    0] train MF loss: 2.24767923, valid loss: 0inf, time 0.10882253 minutes
********************************************************************************
[Epoch 0|Step 1000|Flag 0|Sparsity 0.9896|Params 49971| AUC [0.5208131448951006, 0]]
[Epoch 0|Step 2000|Flag 0|Sparsity 0.9896|Params 49971| AUC [0.5208131448951006, 0]]
[Epoch 1|Step 656|Flag 0|Sparsity 0.9896|Params 49971| AUC [0.5208131448951006, 0]]
[Epoch 1|Step 1656|Flag 0|Sparsity 0.9896|Params 49971| AUC [0.5208131448951006, 0]]
[Epoch 2|Step 312|Flag 0|Sparsity 0.9896|Params 49971| AUC [0.52

In [ ]:
avazu_Deepfm_PEP_retrain = {}
for i, param in enumerate(target_parameters):
    avazu_Deepfm_PEP_retrain[param] =  best_test_results_list[i]

pd.DataFrame(avazu_Deepfm_PEP_retrain).to_csv("avazu_Deepfm_PEP_retrain.csv")

# Train For AutoInt, avazu

In [11]:

torch.cuda.empty_cache()


parser = setup_args()
parser.set_defaults(
    alias='train',
    tensorboard='./tmp/runs/{factorizer}/{data_type}',
    ##########
    ## data ##
    ##########
    data_type='avazu',
    data_path='./data/{data_type}/',
    load_in_queue=False,
    category_only=False,
    rebuild_cache=False,
    eval_res_path='./tmp/res/{factorizer}/{data_type}/{alias}/{epoch_idx}.csv',
    emb_save_path='./tmp/embedding/{factorizer}/{data_type}/{alias}/{num_parameter}',
    ######################
    ## train/test split ##
    ######################
    test_ratio=0.1,
    valid_ratio=1/9,
    ##########################
    ## Devices & Efficiency ##
    ##########################
    use_cuda=True,
    early_stop=10,
    log_interval=1,
    display_interval=1000,
    eval_interval=5,  # 10 epochs between 2 evaluations
    device_ids_test=[0],
    device_id=0,
    batch_size_train=1024,
    batch_size_valid=1024,
    batch_size_test=1024,
    ###########
    ## Model ##
    ###########
    factorizer='fm',
    model='autoint',
    fm_lr=1e-3,
    # Deep
    mlp_dims=[100, 100],
    # AutoInt
    has_residual=True,
    full_part=True,
    num_heads=2,
    num_layers=3,
    att_dropout=0.4,
    atten_embed_dim=64,
    # optimizer setting
    fm_optimizer='adam',
    fm_amsgrad=False,
    fm_eps=1e-8,
    fm_l2_regularization=1e-5,
    fm_betas=(0.9, 0.999),
    fm_grad_clip=100,  # 0.1
    fm_lr_exp_decay=1,
    l2_penalty=0,
    #########
    ## Embeddings//PEP ##
    #########
    latent_dim=32,
    threshold_type='feature_dim',
    g_type='sigmoid',
    gk=1,
    threshold_init=-15,
    candidate_p=[50000, 30000],
)

opt = parser.parse_args(args=[])
opt = vars(opt)

# rename alias

opt['alias'] = '{}_{}_BaseDim{}_bsz{}_lr_{}_optim_{}_thresholdType{}_thres_init{}_{}-{}_l2_penalty{}'.format(
    opt['model'].upper(),
    opt['alias'],
    opt['latent_dim'],
    opt['batch_size_train'],
    opt['fm_lr'],
    opt['fm_optimizer'],
    opt['threshold_type'].upper(),
    opt['threshold_init'],
    opt['g_type'],
    opt['gk'],
    opt['l2_penalty']
)
print(opt['alias'])
random.seed(opt['seed'])
# np.random.seed(opt['seed'])
torch.manual_seed(opt['seed'])
torch.cuda.manual_seed_all(opt['seed'])
engine = Engine(opt)
best_result = engine.train()
print("*"*10 , best_result, "*"*10)



AUTOINT_train_BaseDim32_bsz1024_lr_0.001_optim_adam_thresholdTypeFEATURE_DIM_thres_init-15_sigmoid-1_l2_penalty0
	Num of train records: 32343173
	Num of valid records: 4042896
	Num of test records: 4042898
	Num of fields: 22
	Num of features: 645394
BackBone Embedding Parameters:  20652608
--------------------------------------------------------------------------------
[complete episode  starts!]
Initializing ...
BackBone Embedding Parameters:  20652608
********************************************************************************
Save the initial embedding table
********************************************************************************
[Epoch 0|Step 0|Flag 0|Sparsity 0.0001|Params 20650472| Best AUC 0]
[Epoch 0|Step 1000|Flag 0|Sparsity 0.8931|Params 2208227| Best AUC 0]
[Epoch 0|Step 2000|Flag 0|Sparsity 0.9038|Params 1986537| Best AUC 0]
[Epoch 0|Step 3000|Flag 0|Sparsity 0.9156|Params 1743907| Best AUC 0]
[Epoch 0|Step 4000|Flag 0|Sparsity 0.9267|Params 1514735| Best AUC 0]

# Retrain AutoInt

In [ ]:
target_parameters = [29988, 49982]
best_test_results_list = []
for target_param in target_parameters:

    print(f"retrain for target parameters {target_param}")
    parser = setup_args()
    parser.set_defaults(
        alias='train',
        tensorboard='./tmp/runs/{factorizer}/{data_type}',
        ##########
        ## data ##
        ##########
        data_type='avazu',
        data_path='./data/{data_type}/',
        load_in_queue=False,
        category_only=False,
        rebuild_cache=False,
        eval_res_path='./tmp/res/{factorizer}/{data_type}/{alias}/{epoch_idx}.csv',
        emb_save_path='./tmp/embedding/{factorizer}/{data_type}/{alias}/{num_parameter}',
        ######################
        ## train/test split ##
        ######################
        test_ratio=0.1,
        valid_ratio=1/9,
        ##########################
        ## Devices & Efficiency ##
        ##########################
        use_cuda=True,
        early_stop=40,
        log_interval=1,
        display_interval=1000,
        eval_interval=5,  # 10 epochs between 2 evaluations
        device_ids_test=[0],
        device_id=0,
        batch_size_train=1024,
        batch_size_valid=1024,
        batch_size_test=1024,
        ###########
        ## Model ##
        ###########
        factorizer='fm',
        model='autoint',
        fm_lr=1e-3,
        # Deep
        mlp_dims=[100, 100],
        # AutoInt
        has_residual=True,
        full_part=True,
        num_heads=2,
        num_layers=3,
        att_dropout=0.4,
        atten_embed_dim=64,
        # optimizer setting
        fm_optimizer='adam',
        fm_amsgrad=False,
        fm_eps=1e-8,
        fm_l2_regularization=1e-5,
        fm_betas=(0.9, 0.999),
        fm_grad_clip=100,  # 0.1
        fm_lr_exp_decay=1,
        l2_penalty=0,
        #########
        ## PEP ##
        #########
        latent_dim=32,
        threshold_type='feature_dim',
        g_type='sigmoid',
        gk=1,
        threshold_init=-15,
        retrain_emb_param=target_param,
        re_init=False,
    )

    opt = parser.parse_args(args=[])
    opt = vars(opt)
    opt['alias'] = '{}_{}_BaseDim{}_bsz{}_lr_{}_optim_{}_thresholdType{}_thres_init{}_{}-{}_l2_penalty{}'.format(
        opt['model'].upper(),
        opt['alias'],
        opt['latent_dim'],
        opt['batch_size_train'],
        opt['fm_lr'],
        opt['fm_optimizer'],
        opt['threshold_type'].upper(),
        opt['threshold_init'],
        opt['g_type'],
        opt['gk'],
        opt['l2_penalty']
    )
    print(opt['alias'])
    random.seed(opt['seed'])
    # np.random.seed(opt['seed'])
    torch.manual_seed(opt['seed'])
    torch.cuda.manual_seed_all(opt['seed'])
    engine = Engine(opt)
    best_result = engine.train()

    best_test_results_list.append(best_result)
    


retrain for target parameters 19992
AUTOINT_train_BaseDim32_bsz1024_lr_0.001_optim_adam_thresholdTypeFEATURE_DIM_thres_init-15_sigmoid-1_l2_penalty0
0
	Num of train records: 2400000
	Num of valid records: 300000
	Num of test records: 300000
	Num of fields: 39
	Num of features: 150032
Retrain epoch 19992
BackBone Embedding Parameters:  4801024
--------------------------------------------------------------------------------
[complete episode  starts!]
Initializing ...
Retrain epoch 19992
BackBone Embedding Parameters:  4801024
[Epoch 0|Step 0|Flag 0|Sparsity 0.9958|Params 19992| Best AUC 0]
Evaluate on test ...
********************************************************************************
Test AUC: 0.586419 | Logloss: 0.644200
Evaluate Time 0.09862778333333333 minutes
[Epoch    0] train MF loss: 0.63114500, valid loss: 0inf, time 0.10800515 minutes
********************************************************************************
[Epoch 0|Step 1000|Flag 0|Sparsity 0.9958|Params 19992| Be

d:\pytorchEnv\env\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


--------------------------------------------------------------------------------
[complete episode  starts!]
Initializing ...
Retrain epoch 29984
BackBone Embedding Parameters:  4801024
[Epoch 0|Step 0|Flag 0|Sparsity 0.9938|Params 29984| Best AUC 0]
Evaluate on test ...
********************************************************************************
Test AUC: 0.576816 | Logloss: 0.644201
Evaluate Time 0.13949085 minutes
[Epoch    0] train MF loss: 0.63404930, valid loss: 0inf, time 0.15089083 minutes
********************************************************************************
[Epoch 0|Step 1000|Flag 0|Sparsity 0.9938|Params 29984| Best AUC 0.5768]
[Epoch 0|Step 2000|Flag 0|Sparsity 0.9938|Params 29984| Best AUC 0.5768]
[Epoch 1|Step 656|Flag 0|Sparsity 0.9938|Params 29984| Best AUC 0.5768]
[Epoch 1|Step 1656|Flag 0|Sparsity 0.9938|Params 29984| Best AUC 0.5768]
[Epoch 2|Step 312|Flag 0|Sparsity 0.9938|Params 29984| Best AUC 0.5768]
[Epoch 2|Step 1312|Flag 0|Sparsity 0.9938|Params 

In [ ]:
AutoInt_avazu_PEP_retrain = {}
for i, param in enumerate(target_parameters):
    AutoInt_avazu_PEP_retrain[param] =  best_test_results_list[i]

pd.DataFrame(AutoInt_avazu_PEP_retrain).to_csv("AutoInt_avazu_PEP_retrain.csv")